# Sustained Phonation Feature Extraction Notebook
In this iPython Notebook, we'll show you how to use
our implementation of the features used in <paper citation>
to extract them on your own dataset.

To follow this tutorial, you'll need some audio files to run our
feature extractors on. Ideally, these audio samples each
contain a recording of someone speaking a vowel for a few seconds.

## Step 1: Loading your audio samples

In this part of the notebook, load your audio samples in any way you wish,
the goal is to have them all as Numpy arrays. We'll store them in
a neat little list of `AudioSample` objects which just stores the PCM array,
its sampling rate, and and a string identifier for the sample.

In this example, we suppose that all the audio samples are read from
single WAV files in a folder.

In [ ]:
from features import AudioSample
from pathlib import Path
from typing import List
import librosa
import numpy as np

samples_folder = Path("path/to/samples/folder/")
audio_samples: List[AudioSample] = []
for wav_file in samples_folder.iterdir():
    if not wav_file.suffix == ".wav":
        continue
    else:
        # here, we're loading the samples using librosa
        # TODO : figure out if we should normalize the sampling rate or not
        audio_array, rate = librosa.core.load(str(wav_file), sr=22050,
                                              dtype=np.uint16)
        audio_samples.append(AudioSample(data=audio_array,
                                         rate=rate,
                                         name=wav_file.name))

## Step 2: Configure the feature extractors

In this step, you can tweak, remove of add feature extractors.
Here we've put all the feature extractors from <paper>.
All of their actual implementations are kept in the `features` module contained
in this repository.

In [ ]:
from typing import Dict

# TODO : maybe indicate which feature may take some time to compute (RPDE, DFA, ...)

from features import *
features_dict: Dict[str, SampleProcessor] = {
    "hnr": PraatHNR(),
    "tremor": VocalTremorFeatures(),
    "f0_stats" : AdvancedFundamentalFrequency(),
    "rpde": RPDE(dim=4, tau=35, epsilon=0.12, tmax=5000),
    "dfa": DFA(scale_boundaries=(1.259, 1.559), scale_density=0.03),
    # ... and so on for all features
}

## Step 3: Build and run the extraction

We'll define here a helper function that runs the extraction using the samples
and the selected feature extractors (or processors). Then, we'll run
that function on our audio samples dataset.

Once the feature extraction is complete, all the features computed for all the
samples should be stored in dictionary.

In [ ]:
from collections import defaultdict
import tqdm

def extract(samples_list: List[AudioSample],
          features: Dict[str, SampleProcessor],
          fail_on_error: bool = True):
    """This function takes care of running the extraction,
    and stores the extracted features for each sample in a neat
    little dictionnary"""
    extracted_features = defaultdict(dict)
    for feature_name, processor in features.items():
        for audio_sample in tqdm.tqdm(samples_list):
            try:
                proc_output = processor.process(audio_sample)
            except Exception as e:
                if fail_on_error:
                    raise e
                else:
                    print(f"Got error on sample '{audio_sample.name}':' "
                          f"{type(e).__name__} :{str(e)}', skipping.")
            else:
                # maybe we should use tuples for the dict keys?
                extracted_features[audio_sample.name][feature_name] = proc_output

    return extracted_features



In [ ]:
# then, we run the extraction.
# WARNING: this may (and should) take some time
extracted_features = extract(audio_samples, features_dict, fail_on_error=True)

## Step 4 : Results Analysis

Now, we'll use these features to run a couple of statistical analysis.

In [ ]:
# TODO